In [1]:
!if ! test -f beeradvocate.json.gz; then wget https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz; fi;
# !gzip -d beeradvocate.json.gz

!pip install tqdm

In [2]:
import gzip
from tqdm import tqdm
import numpy as np
import string
from collections import defaultdict
from sklearn import linear_model
import gensim
from gensim.models import Doc2Vec

In [3]:
fpath = "beeradvocate.json.gz"

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

data = []
for l in readGz(fpath):
    data.append(l)
    if len(data) >= 100000:
        break

In [4]:
n = len(data)

dataTrain = data[:int(n*0.8)]
dataVal = data[int(n*0.8):int(n*0.9)]
dataTest = data[int(n*0.9):]

In [5]:
dataTrain[0]

{'beer/name': 'Sausa Weizen',
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'beer/ABV': '5.00',
 'beer/style': 'Hefeweizen',
 'review/appearance': '2.5',
 'review/aroma': '2',
 'review/palate': '1.5',
 'review/taste': '1.5',
 'review/overall': '1.5',
 'review/time': '1234817823',
 'review/profileName': 'stcules',
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.'}

In [6]:
def get_corpus(data):
  for i, d in enumerate(data):
    tokens = gensim.utils.simple_preprocess(d['review/text'])
    yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [7]:
train_corpus = list(get_corpus(dataTrain))
val_corpus = list(get_corpus(dataVal))

In [8]:
print(train_corpus[0])

TaggedDocument<['lot', 'of', 'foam', 'but', 'lot', 'in', 'the', 'smell', 'some', 'banana', 'and', 'then', 'lactic', 'and', 'tart', 'not', 'good', 'start', 'quite', 'dark', 'orange', 'in', 'color', 'with', 'lively', 'carbonation', 'now', 'visible', 'under', 'the', 'foam', 'again', 'tending', 'to', 'lactic', 'sourness', 'same', 'for', 'the', 'taste', 'with', 'some', 'yeast', 'and', 'banana'], [0]>


In [9]:
model = Doc2Vec(vector_size=100, min_count=2, epochs=50)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [10]:
Xtrain = [model.infer_vector(gensim.utils.simple_preprocess(d['review/text'])) for d in dataTrain]
Xval = [model.infer_vector(gensim.utils.simple_preprocess(d['review/text'])) for d in dataVal]

In [11]:
ytrain = [d['review/overall'] for d in dataTrain]
yval   = [d['review/overall'] for d in dataVal]

In [12]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(Xtrain, ytrain)
ypreds = mod.predict(Xval)
correct = [int(yp == y) for yp, y in zip(ypreds, yval)]
acc = sum(correct) / len(correct)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
mod.score(Xval, yval)

0.443

In [14]:
ypreds[:10]

array(['4', '3.5', '4', '4', '4', '3', '4', '4', '4', '4'], dtype='<U3')

In [15]:
Xtrain[0]

array([-0.41563013,  0.07462738, -0.98048615, -1.0502536 ,  0.99023014,
       -0.83772045, -0.3496178 , -1.4450861 ,  0.09863725, -0.09043857,
       -0.8168729 , -0.43389428,  0.43815145, -0.32483727,  0.38028795,
       -1.235194  ,  0.32259545,  0.78997403,  0.6593457 , -0.68907124,
        1.6355003 , -0.19593638,  0.1505881 , -0.42939603, -1.0580109 ,
       -1.5753169 , -1.450085  , -0.8762979 , -0.5797334 ,  0.96016276,
        0.71337205, -0.49275005,  1.3983572 ,  0.51323986, -0.6479618 ,
       -0.23662163,  0.06465844,  0.8781828 , -1.146156  , -0.7381294 ,
        0.06444164,  1.1686393 , -1.0454475 , -0.26545933, -1.2740368 ,
       -0.30584645,  1.1562766 ,  0.1302998 ,  0.809188  ,  0.36150536,
       -0.6671667 , -1.6230819 ,  0.34073788, -0.2661244 ,  0.87624466,
        0.73004043,  1.5364116 ,  0.3695793 , -1.0929552 , -0.16224444,
        0.627256  , -1.3580552 , -0.9250101 , -0.9219991 ,  1.8120254 ,
        1.295882  ,  1.1991459 , -1.1202805 ,  1.3451046 , -0.26